Query EnameStore for price

In [ ]:
!ls final_pre

In [ ]:
from rdkit import Chem

with Chem.SDMolSupplier('final_pre/A71-ROCS-cov.sdf') as sdfh:
    vcs = list(sdfh)

In [ ]:
prices = {} if 'prices' not in globals() else prices 

import requests, time

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

def assign_catalog(name):
    if not name:
        raise ValueError('Empty name')
    if name[0] in 'ZP':
        return 'REALDB'
    if name[0] in 'E':
        return 'BB'
    else:
        raise ValueError(f'Not sure what {name} is')


for mol in vcs[1:]:
    catalog_id = mol.GetProp('_Name').replace('_', '-')
    if catalog_id in prices:
        continue
    catalog = assign_catalog( catalog_id )
    response = requests.get(
            #f'https://new.enaminestore.com/api/v1/catalog/price?id={enamine_code}&cat={db_name}&cur={cur_name}',
            f'https://new.enaminestore.com/api/v1/catalog/price?id={catalog_id}&cat={catalog}&cur=USD',
            headers=headers
    )
    prices[catalog_id] = response.json()
    time.sleep(20)

In [ ]:
catalog_id

In [ ]:
import json

with open('prices.json', 'w') as fh:
    json.dump(prices, fh)

In [ ]:
len(prices.keys())

In [ ]:
vcs[0].SetProp('price', 'List price in USD from EnamineStore')

In [ ]:
def extract_mg(data, NA_value=0.):
    for datum in data['samples']:
        if datum['amount'] != 1.0 or datum['measure'] != 'mg':
            continue
        return datum['price']
    if len(data['samples']) == 0:
        return NA_value
    datum = data['samples'][0]
    if datum['measure'] == 'mg':
        return data['samples'][0]['price'] / datum['amount']
    elif datum['measure'] == 'g':
        return data['samples'][0]['price'] / datum['amount'] / 1_000
    elif datum['measure'] == 'kg':
        return data['samples'][0]['price'] / datum['amount'] / 1_000_000
    else:
        return NA_value

In [ ]:
for mol in vcs[1:]:
    catalog_id = mol.GetProp('_Name').replace('_', '-')
    price = extract_mg(prices[catalog_id])
    print(catalog_id, price)
    mol.SetDoubleProp('price', price)

In [ ]:
with Chem.SDWriter('final_price/A71_fragmenstein_clustered4.sdf') as sdfh:
    for mol in vcs:
        sdfh.write(mol)